In [1]:
import datetime
import time
import sys
sys.path.append(r'/home/hengtian/lucy/factor-analyzer-v4/')
import pymysql
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import multiprocessing
import os
import queue

# from src import data 
# from src import analyzer


import datetime as dt

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn import linear_model
# import cufflinks as cf
# cf.go_offline()

In [2]:
#隔夜涨跌幅
ovngt = pd.read_csv("/home/hengtian/lucy/factor_service/basic_data/ovngt.csv", index_col = 0)

In [54]:
pm_close =pd.read_csv("/home/hengtian/lucy/factor_service/basic_data/minute_1400_open.csv", index_col = 0)

In [3]:
pm_1430 = pd.read_csv("/home/hengtian/lucy/factor_service/basic_data/minute_1430_open.csv", index_col = 0)

In [4]:
am_1030 = pd.read_csv("/home/hengtian/lucy/factor_service/basic_data/minute_1030_open.csv", index_col = 0)

In [104]:
open_price =pd.read_csv("/home/hengtian/lucy/factor_service/basic_data/open_price.csv", index_col = 0)

In [105]:
open_price = open_price['2016':'2021-06-22']

In [44]:
pm_1430

,002218,601799,002667,300374,002574,002859,300138,600747,600234,603083,...,301018,688226,301023,605162,301039,688789,605020,688087,688305,605028
2018-01-02,5.39,52.59,16.87,15.02,7.50,36.65,12.80,2.74,13.77,47.81,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-03,5.50,52.00,16.98,16.59,NaN,36.21,12.92,2.76,NaN,46.49,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-04,5.49,52.59,NaN,17.00,7.67,36.07,12.90,NaN,13.76,46.37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-05,5.43,52.16,16.71,16.74,7.70,35.86,12.81,2.71,NaN,45.08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-08,5.51,52.23,16.52,16.36,7.63,34.97,12.66,2.68,13.72,44.43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-16,4.28,206.38,16.59,12.53,4.28,29.48,14.09,NaN,12.85,14.37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-17,4.29,201.97,15.49,12.50,4.20,30.71,14.11,NaN,12.66,14.75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-18,4.28,214.99,13.81,12.23,4.25,31.01,13.99,NaN,12.74,14.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-21,4.30,229.55,NaN,12.80,4.27,30.63,14.00,NaN,13.19,14.98,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
am_1030=am_1030['2016':'2021-06-22']
am_1030_rv = am_1030[ovlp]
dic1 = dict(zip(am_1030_rv.columns,ovngt_rv.columns))
am_1030_rv = am_1030_rv.rename(dic1,axis = 'columns')

In [61]:
pm_close = pm_close['2016':'2021-06-2']

In [21]:
pm_1430 = pm_1430['2016':'2021-06-22']

In [22]:
pm_1430_rv= pm_1430[ovlp]
dic = dict(zip(pm_1430_rv.columns,ovngt_rv.columns))
pm_1430_rv = pm_1430_rv.rename(dic,axis = 'columns')

In [16]:
ovngt =ovngt['2016':'2021-06-22']

In [9]:
stocks = [col[:6] for col in ovngt.columns]
ovlp = [col for col in stocks if col in pm_1430.columns]
a = [col for col in ovngt.columns if col[:6] in pm_1430.columns]

In [17]:
# pm_close_rv = pm_close[ovlp]
ovngt_rv = ovngt[a]

In [106]:
open_price_rv = open_price[a]

In [12]:
dic = dict(zip(am_1030_rv.columns,ovngt_rv.columns))
am_1030_rv = am_1030_rv.rename(dic,axis = 'columns')

In [126]:
am_1030_er = am_1030_rv/open_price_rv-1

In [257]:
close = pd.read_csv("/home/hengtian/lucy/factor_service/basic_data/close_price.csv", index_col = 0)
close =close["2018":'2021-06-22']

In [258]:
close = close[a]

In [259]:
pm_er = close/pm_1430_rv-1

In [51]:
#隔夜指数收益
bmk_am = am30_rv.mean(axis =1)
#下午指数收益
bmk_pm = span_er.mean(axis=1)

In [40]:
am30 = pd.read_csv("/home/hengtian/kate/chg/am30_chg.csv", index_col = 0)
am30= am30.drop('idx',axis = 1) 

In [45]:
stocks = [col[:6] for col in am30.columns]
ovlp = [col for col in stocks if col in pm_1430.columns]
b= [col for col in am30.columns if col[:6] in pm_1430.columns]

In [48]:
pm_1430_rv= pm_1430[ovlp]
dic = dict(zip(pm_1430_rv.columns,am30.columns))
pm_1430_rv = pm_1430_rv.rename(dic,axis = 'columns')

In [49]:
am_1030_rv = am_1030[ovlp]
dic1 = dict(zip(am_1030_rv.columns,am30.columns))
am_1030_rv = am_1030_rv.rename(dic1,axis = 'columns')

In [46]:
am30_rv = am30[b]

In [50]:
span_er =pm_1430_rv/am_1030_rv-1

In [30]:
ovngt_rv = ovngt_rv['2018':]

In [34]:
model=linear_model.LinearRegression()

In [52]:
#求apm因子值
def task(stock,q):
    i = 20
    o_er = am30_rv[stock]
    p_er = span_er[stock]
    stats = np.full(len(o_er), np.nan)
    while i<len(o_er):
        if len(o_er.iloc[i-20:i].dropna())!= 20 or len(p_er.iloc[i-20:i].dropna()) !=20:
            i+=1
        else:
            s_am = np.array(o_er.iloc[i-20:i])
            s_pm = np.array(p_er.iloc[i-20:i])
            s_bmk_a =np.array(bmk_am.iloc[i-20:i])
            s_bmk_p =np.array(bmk_pm.iloc[i-20:i])
            test = np.concatenate((s_am,s_pm),axis = 0)
            bmk = np.concatenate((s_bmk_a,s_bmk_p),axis = 0)
            model.fit(bmk.reshape(-1,1),test)
            res_a = test[0:20]-model.predict(bmk[0:20].reshape(-1,1))
            res_p = test[20:]-model.predict(bmk[20:].reshape(-1,1))
            resid = res_a-res_p
            stat = np.mean(resid)/(np.std(np.array(resid))/np.sqrt(20))
            stats[i]=stat
            i+=1
    return q.put({stock:stats})

In [53]:
q = multiprocessing.Manager().Queue()
param = []
for stock in span_er.columns:
    param.append((stock,q))
# print(param)
#task
with multiprocessing.Pool(processes =4) as pool:
    pool.starmap(task,param)

#extract
elements = []
while q.qsize():
    elements.append(q.get())

In [54]:
dm = pd.DataFrame(np.empty((len(span_er),span_er.shape[1]),dtype=object),columns = span_er.columns, index = span_er.index)
for i in range(len(elements)):
    stock = list(elements[i].keys())[0]
    val = list(elements[i][stock])
    dm[stock]=val

In [55]:
dm.to_csv("am30_1030_1430.csv")

In [256]:
dm

,000001.XSHE,000002.XSHE,000004.XSHE,000005.XSHE,000006.XSHE,000007.XSHE,000008.XSHE,000009.XSHE,000010.XSHE,000011.XSHE,...,688690.XSHG,688696.XSHG,688698.XSHG,688699.XSHG,688700.XSHG,688777.XSHG,688788.XSHG,688819.XSHG,688981.XSHG,689009.XSHG
2018-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-16,-1.723754,0.423078,NaN,NaN,0.123577,NaN,-1.476084,1.586969,1.531663,-2.082109,...,NaN,NaN,NaN,0.541460,NaN,NaN,NaN,0.494072,1.069869,-1.324656
2021-06-17,-1.626308,0.739126,NaN,NaN,0.380067,NaN,-1.495995,0.456789,1.734889,-1.742180,...,NaN,NaN,NaN,0.242999,NaN,NaN,NaN,0.405752,0.793634,-1.159822
2021-06-18,-2.167141,0.543420,NaN,NaN,0.356482,NaN,-1.186304,1.266477,1.853858,-1.408143,...,NaN,NaN,NaN,0.310084,NaN,-0.703132,NaN,0.684227,-0.604174,-1.371526
2021-06-21,-1.979731,0.407816,NaN,NaN,0.102479,NaN,-1.229095,0.750062,1.120283,-1.457120,...,NaN,NaN,NaN,1.219616,NaN,NaN,NaN,0.515089,-0.549994,-1.048132
